# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>
---

**Lab 07**: Structured Streaming with Files 

**Date**: October 10th 2025

**Student Name**: José Juan Díaz Campos

**Professor**: Pablo Camarillo Ramirez

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import split, col, window, trim
from pyspark.sql.types import StructType, StructField, StringType, TimestampType

spark = SparkSession.builder \
    .appName("Lab07-StructuredStreaming-Files") \
    .master("spark://spark-master:7077") \
    .getOrCreate()
    
sc = spark.sparkContext
sc.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/09 04:39:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/10/09 04:39:35 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
input_path = ("/opt/spark/work-dir/data/logs/input")

streaming_df = spark.readStream \
    .format("text") \
    .load(input_path)

In [3]:
log_parts = split(col("value"), "\\|")

parsed_df = streaming_df.select(
    trim(log_parts.getItem(0)).cast(TimestampType()).alias("timestamp"),
    trim(log_parts.getItem(1)).alias("level"),
    trim(log_parts.getItem(2)).alias("message"),
    trim(log_parts.getItem(3)).alias("server")
)

critical_errors_df = parsed_df.filter(
    (col("level") == "ERROR") & (col("message").contains("500"))
)

error_counts_df = critical_errors_df.groupBy("message").count()

In [4]:
query = error_counts_df.writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()

print("Iniciando stream")
query.awaitTermination(70)

print("Stream detenido.")
query.stop()

Iniciando stream


-------------------------------------------
Batch: 0
-------------------------------------------
+-------+-----+
|message|count|
+-------+-----+
+-------+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+--------------------+-----+
|             message|count|
+--------------------+-----+
|500 Internal Serv...|    2|
+--------------------+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+--------------------+-----+
|             message|count|
+--------------------+-----+
|500 Internal Serv...|    2|
+--------------------+-----+



-------------------------------------------
Batch: 3
-------------------------------------------
+--------------------+-----+
|             message|count|
+--------------------+-----+
|500 Internal Serv...|    2|
+--------------------+-----+



[Stage 10:=================>                                     (62 + 1) / 200]

Stream detenido.


25/10/09 04:40:53 ERROR WriteToDataSourceV2Exec: Data source write support MicroBatchWrite[epoch: 4, writer: ConsoleWriter[numRows=20, truncate=true]] is aborting.
25/10/09 04:40:53 ERROR WriteToDataSourceV2Exec: Data source write support MicroBatchWrite[epoch: 4, writer: ConsoleWriter[numRows=20, truncate=true]] aborted.
[Stage 10:==================>                                    (66 + 1) / 200]